# Data Location Visualisation

This notebook creates a data visualisation, showing the location of each image taken. The visualisation may then be seen by opeining the generated .html file.

In [1]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import folium
import os

In [2]:
dataset = "trashy_dataset"

In [3]:
def get_exif_data(image_path):
    """Extract GPS data from an image."""
    image = Image.open(image_path)
    exif_data = {}
    info = image._getexif()
    if info:
        for tag, value in info.items():
            decoded_tag = TAGS.get(tag, tag)
            if decoded_tag == "GPSInfo":
                gps_data = {}
                for t in value:
                    gps_tag = GPSTAGS.get(t, t)
                    gps_data[gps_tag] = value[t]
                exif_data["GPSInfo"] = gps_data
    return exif_data

def convert_to_decimal(gps_data):
    """Convert GPS coordinates from degrees, minutes, and seconds to decimal."""
    def dms_to_decimal(dms, ref):
        # Check for division by zero
        try:
            degrees = dms[0].numerator / dms[0].denominator
            minutes = dms[1].numerator / dms[1].denominator / 60.0
            seconds = dms[2].numerator / dms[2].denominator / 3600.0
        except ZeroDivisionError:
            return None  # Return None if any part is invalid
        decimal = degrees + minutes + seconds
        return -decimal if ref in ['S', 'W'] else decimal
    
    try:
        lat = dms_to_decimal(gps_data['GPSLatitude'], gps_data['GPSLatitudeRef'])
        lon = dms_to_decimal(gps_data['GPSLongitude'], gps_data['GPSLongitudeRef'])
        if lat is None or lon is None:
            return None
        return lat, lon
    except KeyError:
        return None

In [4]:
image_paths = [os.path.join(dataset, file) for file in os.listdir(dataset) if file.lower().endswith(('.jpg', '.jpeg'))]
map_center = [35.9375, 14.3754]

m = folium.Map(location=map_center, zoom_start=11.4)

# Process each image and add markers if GPS data exists
for image_path in image_paths:
    exif_data = get_exif_data(image_path)
    if "GPSInfo" in exif_data:
        gps_coords = convert_to_decimal(exif_data["GPSInfo"])
        if gps_coords:
            folium.Marker(location=gps_coords, popup=image_path).add_to(m)
        else:
            print(f"Skipped image due to invalid GPS data: {image_path}")
    else:
        print(f"No GPS data found for image: {image_path}")

# Save the map
m.save("map_with_images.html")

Skipped image due to invalid GPS data: trashy_dataset/MSIDA_11.jpg
Skipped image due to invalid GPS data: trashy_dataset/MSIDA_39.jpg
No GPS data found for image: trashy_dataset/IMG_0216.JPG
No GPS data found for image: trashy_dataset/462554911_1659161761652084_4338913442653225430_n.jpg
No GPS data found for image: trashy_dataset/IMG_0202.JPG
No GPS data found for image: trashy_dataset/462561565_536747682504788_5409415532377051106_n.jpg
No GPS data found for image: trashy_dataset/462569815_867731755523476_1871443311804233879_n.jpg
No GPS data found for image: trashy_dataset/IMG_0203.JPG
No GPS data found for image: trashy_dataset/IMG_0217.JPG
No GPS data found for image: trashy_dataset/462562806_1346095680101738_2716219390872206992_n.jpg
Skipped image due to invalid GPS data: trashy_dataset/MSIDA_38.jpg
Skipped image due to invalid GPS data: trashy_dataset/MSIDA_10.jpg
No GPS data found for image: trashy_dataset/IMG_1782 2.jpg
Skipped image due to invalid GPS data: trashy_dataset/MSIDA